Riding the Wave

Finite Volume Method:

--Very general and don't require grid structure
--Gives conservative discretization by using directly the conservation laws in integral form

Conservation Law for a quantity e:

$$\frac{\partial}{\partial t} \int_{CV} e dV + \oint_{CS} \vec{F} \cdot d\vec{A} = 0 $$

Define $e_i$ as the integral across CV on the cell with center at $x_i$

$$e_i = \frac{1}{\Delta x} \int_{x_i}^{x_i + \frac{\Delta x}{2}} e(x,t)dx$$

General Conservation Law becomes:

$$\frac{\partial}{\partial t} e_i + \frac{1}{\Delta x}[F(x_{i+\frac{1}{2}},t) - F(x_{i-\frac{1}{2}},t)] = 0$$

We will need to apporximate the flux term at the cell edges

Godunov's Method

The numerical flux on $x_{i+1/2}$ is:

$$F_{i+1/2} = \frac{1}{\Delta t}\int_{t^n}^{t^{n+1}} F(e(x_{i+1/2}, t))dt$$

Rusanov Flux (Lax-Friedrichs):

$$F_{i+1/2} = \frac{1}{2}[F(e_L) + F(e_R)] - \frac{1}{2}max|F'(e)|(e_R-e_L)$$

Where $F'(e)$ is the Jacobian flux function. Rieman solutions at each cell boundary do not interact if $max|F'(e)|\leq \frac{\Delta x}{\Delta t}$

$$F_{i+1/2} = \frac{1}{2} \left(F(e_i)+F(e_{i+1}) - \frac{\Delta x}{\Delta t}(e_{i+1} - e_i) \right)$$



In [ ]:
#Let's Try it

